In [3]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import seaborn as sns
warnings.simplefilter("ignore")

In [1]:
!pip uninstall keras
!pip install keras==2.2.4

import os

os.environ["KERAS_BACKEND"] = "theano"
import keras.backend
keras.backend.set_image_dim_ordering('th')

Found existing installation: Keras 2.2.4
Uninstalling Keras-2.2.4:
  Would remove:
    /usr/local/lib/python3.10/dist-packages/Keras-2.2.4.dist-info/*
    /usr/local/lib/python3.10/dist-packages/docs/*
    /usr/local/lib/python3.10/dist-packages/keras/*
  Would not remove (might be manually added):
    /usr/local/lib/python3.10/dist-packages/docs/conf.py
Proceed (Y/n)? ERROR: Operation cancelled by user


Using Theano backend.


ModuleNotFoundError: No module named 'theano'

In [ ]:
!pip install pandas
!file -i '/HouseListings-Top45Cities-10292023-kaggle.csv'
df = pd.read_csv('/HouseListings-Top45Cities-10292023-kaggle.csv', encoding='latin-1')
df.head()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Data Features

In [ ]:
df.describe()


In [ ]:
df.info()

In [ ]:
plt.figure(figsize=(12,4))
sns.histplot(df["Price"])

plt.show()

In [ ]:
tmp_df = df.groupby("City").agg({"Price":"median","Median_Family_Income":"mean"}).sort_values("Price",ascending=False)
plt.figure(figsize=(15,30))

plt.subplot(311)
plt.xticks(rotation=90)
sns.barplot(x=tmp_df.index, y=tmp_df["Price"])

plt.subplot(312)
plt.xticks(rotation=90)
sns.barplot(x=tmp_df.index, y=tmp_df["Median_Family_Income"])

plt.subplot(313)
plt.xticks(rotation=90)
sns.barplot(x=tmp_df.index, y=tmp_df["Price"]/tmp_df["Median_Family_Income"])
plt.ylabel("Price/Income")

plt.show()

In [ ]:
plt.figure(figsize=(12,12))

plt.subplot(211)
sns.scatterplot(x=df["Number_Beds"],y=df["Price"])

plt.subplot(212)
sns.scatterplot(x=df["Number_Baths"],y=df["Price"])
plt.show()

we add an indicator of weather the property is an apartment or not by the number of the bedroom, properties with 3 and more bedrooms are identified as apartments.

In [ ]:
df['Is_House'] = (df['Number_Beds'] >= 3).astype(int)
df.head()

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(14, 6))

# Boxplot for Price vs. Number of Beds
sns.boxplot(x='Number_Beds', y='Price', data=df, ax=axes[0])
axes[0].set_title('Price Distribution by Number of Bedrooms')
axes[0].set_xlabel('Number of Bedrooms')
axes[0].set_ylabel('Price')

# Boxplot for Price vs. Number of Baths
sns.boxplot(x='Number_Baths', y='Price', data=df, ax=axes[1])
axes[1].set_title('Price Distribution by Number of Bathrooms')
axes[1].set_xlabel('Number of Bathrooms')
axes[1].set_ylabel('Price')

# Adjust layout and display the plots
plt.tight_layout()
plt.show()

Correlation Matrix \\
Prior to running a regression, we need to ensure that our independent variables are not strongly correlated with one and other. Any strong correlations amongst the independent variables may result in endogeneity and problems of multicollinearity

In [ ]:
# Calculate the correlation matrix
corr_matrix = df.corr()

plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, fmt=".2f", cmap='coolwarm', square=True, linewidths=.5, cbar_kws={"shrink": .5})

plt.title('Correlation Matrix of DataFrame Variables')
plt.show()

Summary Statistics \\
We can then proceed with describing the features of the dataset using summary statistics, such as; mean, median, mode and value counts for each of feature.

In [ ]:
summary_stats = df.describe()
print(summary_stats)


Histogram of response variable\\
A downward trend is clearly visible



In [ ]:
# Generate a histogram of the 'Price' column

plt.figure(figsize=(10, 6))
plt.hist(df['Price'], bins=50, alpha=0.75)

plt.title('Histogram of Property Prices')
plt.xlabel('Price')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()

# Methology


1.   Research Question
2.   Priors Set Up
3.   Model Construction
4.   Sampling
5.   Posterior Analysis





# 1. Research Question:


*   "How does the location (city) influence the housing prices, after controlling for house-specific features such as Number_Beds, Number_Baths, and Median_Family_Income, in the top 45 cities?"



# 2.  Priors Set Up


In [ ]:
import pymc as pm
import pandas as pd


city_indices = pd.Categorical(df['City']).codes
num_cities = len(df['City'].unique())

with pm.Model() as hierarchical_model:
    # Hyperpriors for the global model parameters
    mu_alpha = pm.Normal('mu_alpha', mu=0, sigma=10)
    sigma_alpha = pm.HalfNormal('sigma_alpha', sigma=10)

    # Assuming no prior knowledge about the impact of number of beds and baths
    mu_beta_beds = pm.Normal('mu_beta_beds', mu=0, sigma=10)
    sigma_beta_beds = pm.HalfNormal('sigma_beta_beds', sigma=10)

    mu_beta_baths = pm.Normal('mu_beta_baths', mu=0, sigma=10)
    sigma_beta_baths = pm.HalfNormal('sigma_beta_baths', sigma=10)

    # Assuming some knowledge about the positive relation between income and housing prices
    mu_beta_income = pm.Normal('mu_beta_income', mu=0, sigma=1)
    sigma_beta_income = pm.HalfNormal('sigma_beta_income', sigma=1)

    # Random intercepts and slopes for each city
    alpha = pm.Normal('alpha', mu=mu_alpha, sigma=sigma_alpha, shape=num_cities)
    beta_beds = pm.Normal('beta_beds', mu=mu_beta_beds, sigma=sigma_beta_beds, shape=num_cities)
    beta_baths = pm.Normal('beta_baths', mu=mu_beta_baths, sigma=sigma_beta_baths, shape=num_cities)
    beta_income = pm.Normal('beta_income', mu=mu_beta_income, sigma=sigma_beta_income, shape=num_cities)

    # Model error
    sigma_y = pm.HalfNormal('sigma_y', sigma=1)

    # Expected value of outcome
    price_est = (alpha[city_indices] +
                 beta_beds[city_indices] * df['Number_Beds'] +
                 beta_baths[city_indices] * df['Number_Baths'] +
                 beta_income[city_indices] * df['Median_Family_Income'])

    # Likelihood (sampling distribution) of observations
    price_obs = pm.Normal('price_obs', mu=price_est, sigma=sigma_y, observed=df['Price'])

# Model sampling
with hierarchical_model:
    trace = pm.sample(1000, tune=1000, target_accept=0.9)

# 3. Construct a Hierarchical Model



In [ ]:
import pymc3 as pm
import pandas as pd
import numpy as np
import theano.tensor as tt

# Create an index for each city
city_indices = pd.Categorical(df['City']).codes
num_cities = len(np.unique(city_indices))

with pm.Model() as hierarchical_model:
    # Hyperpriors for the global model parameters
    mu_alpha = pm.Normal('mu_alpha', mu=0, sigma=10)
    sigma_alpha = pm.HalfNormal('sigma_alpha', sigma=10)

    mu_beta_beds = pm.Normal('mu_beta_beds', mu=0, sigma=10)
    sigma_beta_beds = pm.HalfNormal('sigma_beta_beds', sigma=10)

    mu_beta_baths = pm.Normal('mu_beta_baths', mu=0, sigma=10)
    sigma_beta_baths = pm.HalfNormal('sigma_beta_baths', sigma=10)

    mu_beta_income = pm.Normal('mu_beta_income', mu=0, sigma=1)
    sigma_beta_income = pm.HalfNormal('sigma_beta_income', sigma=1)

    # City-specific random intercepts and slopes
    alpha = pm.Normal('alpha', mu=mu_alpha, sigma=sigma_alpha, shape=num_cities)
    beta_beds = pm.Normal('beta_beds', mu=mu_beta_beds, sigma=sigma_beta_beds, shape=num_cities)
    beta_baths = pm.Normal('beta_baths', mu=mu_beta_baths, sigma=sigma_beta_baths, shape=num_cities)
    beta_income = pm.Normal('beta_income', mu=mu_beta_income, sigma=sigma_beta_income, shape=num_cities)

    # Standard deviation of model error
    sigma_y = pm.HalfNormal('sigma_y', sigma=1)

    # Expected value (mean) of outcome
    price_est = (alpha[city_indices] +
                 beta_beds[city_indices] * df['Number_Beds'] +
                 beta_baths[city_indices] * df['Number_Baths'] +
                 beta_income[city_indices] * df['Median_Family_Income'])

    # Data likelihood
    price_obs = pm.Normal('price_obs', mu=price_est, sigma=sigma_y, observed=df['Price'])

    # Posterior sampling
    trace = pm.sample(1000, tune=1000, chains=4, cores=1, target_accept=0.9)

# After the model has run, can do various posterior checks like
# pm.traceplot(trace)
# pm.summary(trace)

# 4. Sampling

In [ ]:
with hierarchical_model:
    # Sampling from the posterior distribution
    trace = pm.sample(1000, tune=1000, chains=4, cores=1, target_accept=0.9)

    # Save the trace for later analysis
    pm.save_trace(trace, 'my_hierarchical_model_trace')

# 5. Posterior Analysis

In [ ]:
#Trace Plots
pm.traceplot(trace)

#Summary Statistics
summary = pm.summary(trace)
print(summary)

#Posterior Predictive Checks
posterior_preds = pm.sample_posterior_predictive(trace, model=hierarchical_model)

#Convergence Diagnostics
pm.rhat(trace)

# ？

In [ ]:
# Define functions
def smryMCMC_HD(codaSamples, compVal=None, saveName=None):
    summaryInfo = pd.DataFrame()
    mcmcMat = np.array(codaSamples, chains=True)
    paramName = codaSamples.columns
    for pName in paramName:
        if pName in compVal.columns:
            if not np.isnan(compVal[pName]):
                summaryInfo = summaryInfo.append(summarizePost(paramSampleVec=mcmcMat[pName], compVal=float(compVal[pName])))
            else:
                summaryInfo = summaryInfo.append(summarizePost(paramSampleVec=mcmcMat[pName]))
        else:
            summaryInfo = summaryInfo.append(summarizePost(paramSampleVec=mcmcMat[pName]))
    summaryInfo.index = paramName

    if saveName is not None:
        summaryInfo.to_csv(saveName + "SummaryInfo.csv")

    return summaryInfo

In [ ]:
def plotMCMC_HD(codaSamples, data, xName="x", yName="y", showCurve=False, pairsPlot=False, compVal=None, saveName=None, saveType="jpg"):
    y = data[yName]
    x = data[xName].values
    mcmcMat = np.array(codaSamples, chains=True)
    chainLength = mcmcMat.shape[0]
    zbeta0 = mcmcMat["zbeta0"]
    zbeta = mcmcMat.filter(regex="^zbeta$|^zbeta\\[")
    if x.ndim == 1:
        zbeta = np.reshape(zbeta, (zbeta.shape[0], 1))
    zVar = mcmcMat["zVar"]
    beta0 = mcmcMat["beta0"]
    beta = mcmcMat.filter(regex="^beta$|^beta\\[")
    if x.ndim == 1:
        beta = np.reshape(beta, (beta.shape[0], 1))
    tau = mcmcMat["tau"]
    pred1 = mcmcMat["pred[1]"]
    pred2 = mcmcMat["pred[2]"]
    pred3 = mcmcMat["pred[3]"]
    pred4 = mcmcMat["pred[4]"]
    pred5 = mcmcMat["pred[5]"]

    YcorX = np.corrcoef(y, x)[0, 1]
    Rsq = np.dot(zbeta, np.reshape(YcorX, (1, 1)))

    if pairsPlot:
        nPtToPlot = 1000
        plotIdx = np.floor(np.arange(0, chainLength, chainLength / nPtToPlot)).astype(int)
        fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(14, 8))
        axes = axes.flatten()
        for i, ax in enumerate(axes):
            if i < zbeta.shape[1]:
                ax.plot(mcmcMat.iloc[plotIdx, i], mcmcMat.iloc[plotIdx, i + zbeta.shape[1]], 'o', color='skyblue')
                ax.set_xlabel("beta[{}]".format(i))
                ax.set_ylabel("beta[{}]".format(i + zbeta.shape[1]))
                ax.set_title("Posterior Pairs")
                ax.text(0.5, 0.5, "r = {:.2f}".format(np.corrcoef(mcmcMat.iloc[plotIdx, i], mcmcMat.iloc[plotIdx, i + zbeta.shape[1]])[0, 1]), transform=ax.transAxes)
        if saveName is not None:
            plt.savefig(saveName + "PostPairs." + saveType)
        plt.show()

    def decideOpenGraph(panelCount, saveName, finished=False, nRow=2, nCol=3):
        if finished:
            if saveName is not None:
                plt.savefig(saveName + str((panelCount - 1) // (nRow * nCol)) + "." + saveType)
            panelCount = 1
            return panelCount
        else:
            if panelCount % (nRow * nCol) == 1:
                if panelCount > 1 and saveName is not None:
                    plt.savefig(saveName + str(panelCount // (nRow * nCol)) + "." + saveType)
                plt.figure(figsize=(nCol * 7.0 / 3, nRow * 2.0))
                plt.subplots_adjust(wspace=0.4, hspace=0.4)
            panelCount += 1
            return panelCount